In [2]:
import pandas as pd

In [47]:
all_mutations = pd.read_csv('/home/kin672/gentb-summer22/Mutations/all_mutations.csv')
mutation_count = pd.read_csv('/home/kin672/gentb-summer22/Mutations/Mutation_Count.csv')
full_db = pd.read_csv('/home/kin672/gentb-summer22/Creating Summaries from Relevant Files/7-18 Output/full_df_7-18.csv')

,isolate,drug,confidence,gene,genome_index,GenTB Mutation,WHO Variant
0,TEST0248,inh,1) Assoc w R,katG,2155168,SNP_CN_2155168_C944G_S315T_katG,katG_S315T
1,TEST0248,rif,1) Assoc w R,rpoB,761155,SNP_CN_761155_C1349T_S450L_rpoB,rpoB_S450L
2,TEST0248,emb,1) Assoc w R,embB,4248003,SNP_CN_4248003_A1490G_Q497R_embB,embB_Q497R
3,TEST0248,amk,1) Assoc w R,rrs,1473246,SNP_N_1473246_A1401G_rrs,rrs_a1401g
4,TEST0248,cap,1) Assoc w R,rrs,1473246,SNP_N_1473246_A1401G_rrs,rrs_a1401g
...,...,...,...,...,...,...,...
47235,TB-111-16552-18_S32_R.all,eth,3) Uncertain significance,ethA,4326632,SNP_CN_4326632_T842G_H281P_ethA,ethA_H281P
47236,TB-111-17896-18_S97_R.all,str,5) Not assoc w R,rpsL,781395,SNP_P_781395_T165C_promoter-rpsL,rpsL_t-165c
47237,TB-111-19509-18_S167_R.all,str,5) Not assoc w R,rpsL,781395,SNP_P_781395_T165C_promoter-rpsL,rpsL_t-165c
47238,TB-111-18567-18-138_S44_R.all,str,3) Uncertain significance,gid,4407995,SNP_CN_4407995_T208G_S70R_gid,gid_S70R


In [ ]:
# Summarize mutation identities for each isolate
num_mutations_per_drug = {'Isolate':[], 'total':[], 'rif':[], 'inh':[], 'emb':[], 'pza':[], 'str':[], 'cap':[], 'amk':[], 'cip':[], 'kan':[], 'levo':[], 'oflx':[], 'pas':[], 'eth':[], 'unknown':[]}

for isolate in full_db.ID.unique():
    num_mutations_per_drug['Isolate'].append(isolate)
    if isolate in mutation_count.Isolate.unique():
        num_total_mutations = mutation_count.loc[mutation_count['Isolate'] == isolate, 'Num_Mutations'].values[0]
        rif = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'rif')])
        inh = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'inh')])
        emb = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'emb')])
        pza = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'pza')])
        strep = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'str')])
        cap = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'cap')])
        amk = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'amk')])
        cip = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'cip')])
        kan = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'kan')])
        levo = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'levo')])
        oflx = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'oflx')])
        pas = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'pas')])
        eth = len(all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == 'eth')])
        not_specified = num_total_mutations - np.sum([rif, inh, emb, pza, strep, cap, amk, cip, kan, levo, oflx, pas, eth]) 
        total = num_total_mutations
    else:
        total, rif, inh, emb, pza, strep, cap, amk, cip, kan, levo, oflx, pas, eth, not_specified = 0
    
    num_mutations_per_drug['total'].append(total)
    num_mutations_per_drug['rif'].append(rif)
    num_mutations_per_drug['inh'].append(inh)
    num_mutations_per_drug['emb'].append(emb)
    num_mutations_per_drug['pza'].append(pza)
    num_mutations_per_drug['str'].append(strep)
    num_mutations_per_drug['cap'].append(cap)
    num_mutations_per_drug['amk'].append(amk)
    num_mutations_per_drug['cip'].append(cip)
    num_mutations_per_drug['kan'].append(kan)
    num_mutations_per_drug['levo'].append(levo)
    num_mutations_per_drug['oflx'].append(oflx)
    num_mutations_per_drug['pas'].append(pas)
    num_mutations_per_drug['eth'].append(eth)
    num_mutations_per_drug['unknown'].append(not_specified)
    
pd.DataFrame(num_mutations_per_drug).to_csv('/home/kin672/gentb-summer22/Mutations/Num_Mutations_Per_Drug.csv', index = False)

    


In [51]:
# Function that produces confidence db for a single drug
# Product db includes (for each isolate): Resistance, Lineage, # of WHO-identified mutations, # of high confidence mutations, # low confidence
def summarize_mutations(drug, all_mutations = all_mutations, mutation_count = mutation_count):
    db = {'Isolate':[], 'Lineage':[], 'Resistant':[], 'WHO-Identified':[], 'High_Confidence':[], 'Low_Confidence':[]}
    
    # for isolate in full_db.isolate.unique():
    for isolate in ['TB-111-16552-18_S32_R.all']:
        lineage = full_db.loc[(full_db['ID'] == isolate) & (full_db['Drug'] == 'rif'), 'Lineage'].values[0]
        resistant = full_db.loc[(full_db['ID'] == isolate) & (full_db['Drug'] == 'rif'), 'Resistant'].values[0]
        db['Isolate'].append(isolate)
        db['Lineage'].append(lineage)
        db['Resistant'].append(resistant)
        
        # If Isolate has any WHO-identified mutations, classify them
        if isolate in all_mutations.isolate.unique():
            WHO_mutations = all_mutations[(all_mutations['isolate'] == isolate) & (all_mutations['drug'] == drug)]
            num_WHO_mutations = len(WHO_mutations)
            num_high_confidence = len(WHO_mutations[(WHO_mutations['confidence'] == '1) Assoc w R') | (WHO_mutations['confidence'] == '2) Assoc w R - Interim')])
            num_low_confidence = num_WHO_mutations - num_high_confidence
            
            db['WHO-Identified'].append(num_WHO_mutations)
            db['High_Confidence'].append(num_high_confidence)
            db['Low_Confidence'].append(num_low_confidence)
        else:
            num_WHO_mutations, num_high_confidence, num_low_confidence = 0
            db['WHO-Identified'].append(num_WHO_mutations)
            db['High_Confidence'].append(num_high_confidence)
            db['Low_Confidence'].append(num_low_confidence)
            
    return(pd.DataFrame(db))
    

In [ ]:
for drug in full_db.Drug.unique():
    pd.DataFrame(summarize_mutations(drug)).to_csv('/home/kin672/gentb-summer22/Mutations/Breakdown Per Drug/' + str(drug) + '_summary_of_mutations.csv', index = False)


In [8]:
pd.read_csv('/home/kin672/gentb-summer22/Mutations/Mutation_Info_Martin.csv')


,syn,ntpos,coding,cref,cpos,cver,amino,aref,apos,aver,noncode,gene,rgene,Isolate,Full Mutation,mode,codes
0,CN,2155168,C944G,C,944,G,S315T,S,315.0,T,NaN,katG,NaN,TEST0248,SNP_CN_2155168_C944G_S315T_katG,NaN,NaN
1,CN,761155,C1349T,C,1349,T,S450L,S,450.0,L,NaN,rpoB,NaN,TEST0248,SNP_CN_761155_C1349T_S450L_rpoB,NaN,NaN
2,CN,4248003,A1490G,A,1490,G,Q497R,Q,497.0,R,NaN,embB,NaN,TEST0248,SNP_CN_4248003_A1490G_Q497R_embB,NaN,NaN
3,N,1473246,A1401G,A,1401,G,NaN,NaN,NaN,NaN,NaN,NaN,rrs,TEST0248,SNP_N_1473246_A1401G_rrs,NaN,NaN
4,CN,7582,A281G,A,281,G,D94G,D,94.0,G,NaN,gyrA,NaN,TEST0248,SNP_CN_7582_A281G_D94G_gyrA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65255,N,1474001,C344T,C,344,T,NaN,NaN,NaN,NaN,NaN,NaN,rrl,TB-111-19509-18_S167_R.all,SNP_N_1474001_C344T_rrl,NaN,NaN
65256,CN,4247145,C632T,C,632,T,S211F,S,211.0,F,NaN,embB,NaN,TB-111-19509-18_S167_R.all,SNP_CN_4247145_C632T_S211F_embB,NaN,NaN
65257,CN,4407995,T208G,T,208,G,S70R,S,70.0,R,NaN,gid,NaN,TB-111-18567-18-138_S44_R.all,SNP_CN_4407995_T208G_S70R_gid,NaN,NaN
65258,P,781395,T165C,T,165,C,NaN,NaN,NaN,NaN,promoter,rpsL,NaN,TB-111-18567-18-138_S44_R.all,SNP_P_781395_T165C_promoter-rpsL,NaN,NaN


In [7]:
pd.read_csv('/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv')


,Unnamed: 0,drug,genome_index,confidence,gene,variant
0,0,AMI,4268928,3) Uncertain significance,aftB,aftB_c-92t
1,1,AMI,4268964,3) Uncertain significance,aftB,aftB_g-128t
2,2,AMI,619747,3) Uncertain significance,ccsA,ccsA_c-144a
3,3,AMI,619700,3) Uncertain significance,ccsA,ccsA_c-191t
4,4,AMI,619795,3) Uncertain significance,ccsA,ccsA_c-96g
...,...,...,...,...,...,...
17528,17567,STM,3568779,5) Not assoc w R,whiB7,whiB7_t-100c
17529,17568,STM,4407620,5) Not assoc w R,gid,gid_Y195H
17530,17569,STM,3568488,5) Not assoc w R,whiB7,whiB7_192_del_1_gc_g
17531,17570,STM,1472337,5) Not assoc w R,rrs,rrs_c492t
